In [8]:
from qiime2 import Artifact
from qiime2.plugins.cutadapt.methods import trim_paired

demuxed_seqs = Artifact.load("demux.qza")
trimmed_seqs = trim_paired(demuxed_seqs, cores=8,
                           front_f=["AATGATACGGCGACCACCGAGATCTACACGCTNNNNNNNNNNNNTATGGTAATTGTGTGYCAGCMGCCGCGGTAA"],
                           front_r=["CAAGCAGAAGACGGCATACGAGATAGTCAGCCAGCCGGACTACNVGGGTWTCTAAT"])

trimmed_seqs.trimmed_sequences.save("dada2/trimmed_seqs.qza")

Running external command line application. This may print messages to stdout and/or stderr.
The commands to be run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: cutadapt --cores 8 --error-rate 0.1 --times 1 --overlap 3 -o /var/folders/k7/wbr70tj56_n4y6h66kszvm2r0000gq/T/q2-CasavaOneEightSingleLanePerSampleDirFmt-hezu__sd/BS-1_0_L001_R1_001.fastq.gz -p /var/folders/k7/wbr70tj56_n4y6h66kszvm2r0000gq/T/q2-CasavaOneEightSingleLanePerSampleDirFmt-hezu__sd/BS-1_1_L001_R2_001.fastq.gz --front AATGATACGGCGACCACCGAGATCTACACGCTNNNNNNNNNNNNTATGGTAATTGTGTGYCAGCMGCCGCGGTAA -G CAAGCAGAAGACGGCATACGAGATAGTCAGCCAGCCGGACTACNVGGGTWTCTAAT /var/folders/k7/wbr70tj56_n4y6h66kszvm2r0000gq/T/qiime2-archive-vbrtelja/b057d45e-ed38-4056-88e8-a45fd5ccb7d7/data/BS-1_0_L001_R1_001.fastq.gz /var/folders/k7/wbr70tj56_n4y6h66kszvm2r0000gq/T/qiime2-archive-vbrtelja/b057d45e-ed38-4056-88e8-a45fd5ccb7d7/data/BS-1_1_L001_R2_001.fastq.gz


Command:

'dada2/trimmed_seqs.qza'

In [13]:
from qiime2 import Artifact
from qiime2.plugins.dada2.methods import denoise_paired

#trimmed_seqs = Artifact.load("dada2/trimmed_seqs.qza")
denoised = denoise_paired(trimmed_seqs, 
                         trunc_len_f = 200, trunc_len_r = 180,
                         max_ee = 4,
                         trunc_q = 2, chimera_method = "pooled",
                         n_threads = 0, n_reads_learn = 100000)

denoised.table.save("dada2/freqs.qza")
denoised.representative_sequences.save("dada2/seqs.qza")


Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada_paired.R /var/folders/k7/wbr70tj56_n4y6h66kszvm2r0000gq/T/tmpof_4hy2q/forward /var/folders/k7/wbr70tj56_n4y6h66kszvm2r0000gq/T/tmpof_4hy2q/reverse /var/folders/k7/wbr70tj56_n4y6h66kszvm2r0000gq/T/tmpof_4hy2q/output.tsv.biom /var/folders/k7/wbr70tj56_n4y6h66kszvm2r0000gq/T/tmpof_4hy2q/track.tsv /var/folders/k7/wbr70tj56_n4y6h66kszvm2r0000gq/T/tmpof_4hy2q/filt_f /var/folders/k7/wbr70tj56_n4y6h66kszvm2r0000gq/T/tmpof_4hy2q/filt_r 200 180 0 0 4 2 pooled 1.0 0 100000



'seqs.qza'

In [1]:
from qiime2 import Artifact
from qiime2.plugins import feature_table
table = Artifact.load("freqs.qza")
feat_tab = feature_table.visualizers.summarize(table)
feat_tab.visualization

<visualization: Visualization uuid: 2f55daa3-a92b-407f-b304-3221de414b71>

In [3]:
from qiime2.plugins.vsearch.pipelines import cluster_features_open_reference
from qiime2 import Artifact

table = Artifact.load("dada2/freqs.qza")
seqs = Artifact.load("dada2/seqs.qza")
otus_97 = Artifact.load("97_otus.qza")
clustered_tab = cluster_features_open_reference(table=table, sequences=seqs, 
                                                reference_sequences=otus_97, perc_identity=0.97, strand="both", threads=8)

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --usearch_global /var/folders/k7/wbr70tj56_n4y6h66kszvm2r0000gq/T/tmpsyuvyxsl --id 0.97 --db /var/folders/k7/wbr70tj56_n4y6h66kszvm2r0000gq/T/qiime2-archive-t3fxfw9u/30ff8d4b-5973-4230-b69c-a8c5504bf428/data/dna-sequences.fasta --uc /var/folders/k7/wbr70tj56_n4y6h66kszvm2r0000gq/T/tmp5gi4w8sc --strand both --qmask none --notmatched /var/folders/k7/wbr70tj56_n4y6h66kszvm2r0000gq/T/tmpoxyqslio --threads 8

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: vsearch --sortbysize /var/folders/k7/wbr70tj56_n4y6h66kszvm2r0000gq/T/tmpoxyqslio --xsize --output /var/folders/k7/wbr70

In [5]:
clustered_tab.clustered_table.save("dada2/clustered_tab.qza")
clustered_tab.clustered_sequences.save("dada2/otu_sequences.qza")

'dada2/otu_sequences.qza'

In [6]:
from qiime2 import Artifact
from qiime2.plugins import feature_table
table = Artifact.load("dada2/clustered_tab.qza")
feat_tab = feature_table.visualizers.summarize(table)
feat_tab.visualization

<visualization: Visualization uuid: 981c5085-2970-4b36-b018-c3dbb8164066>

## Taxonomy Assignment

In [10]:
%%bash
qiime tools import --type 'FeatureData[Taxonomy]' --input-format HeaderlessTSVTaxonomyFormat  --input-path gg_13_8_otus/taxonomy/97_otu_taxonomy.txt --output-path ref-taxonomy.qza

Imported gg_13_8_otus/taxonomy/97_otu_taxonomy.txt as HeaderlessTSVTaxonomyFormat to ref-taxonomy.qza


In [14]:
from qiime2 import Artifact
from qiime2.plugins import feature_classifier

otus_97 = Artifact.load("97_otus.qza")
ref_tax = Artifact.load("ref-taxonomy.qza")

extracted_seqs = feature_classifier.methods.extract_reads(sequences = otus_97,
                                                         f_primer = "GTGYCAGCMGCCGCGGTAA",
                                                         r_primer = "GGACTACNVGGGTWTCTAAT",
                                                         max_length = 400,
                                                         min_length = 100)

classify = feature_classifier.methods.fit_classifier_naive_bayes(extracted_seqs.reads, ref_tax)
classify.classifier.save("dada2/nbclass.qza")

/Users/haneylab/miniconda3/envs/qiime2-2019.1/lib/python3.6/site-packages/q2_feature_classifier/classifier.py:101: UserWarning: The TaxonomicClassifier artifact that results from this method was trained using scikit-learn version 0.20.2. It cannot be used with other versions of scikit-learn. (While the classifier may complete successfully, the results will be unreliable.)
  warnings.warn(warning, UserWarning)


'dada2/nbclass.qza'

In [27]:
from qiime2 import Artifact
from qiime2.plugins.feature_classifier.methods import classify_sklearn

classifier = Artifact.load("dada2/nbclass.qza")
seqs = Artifact.load("dada2/otu_sequences.qza")


tax_tab = classify_sklearn(seqs, classifier, n_jobs = -1)
tax_tab.classification.save("dada2/tax_table.qza")

'dada2/tax_table.qza'

In [28]:
%%bash
#export otu table
qiime tools export --input-path dada2/clustered_tab.qza --output-path phyloseq
biom convert --to-tsv -i phyloseq/feature-table.biom -o phyloseq/otu_table.tsv

#export sequences
qiime tools export --input-path dada2/seqs.qza --output-path phyloseq

#export taxonomy
qiime tools export --input-path dada2/tax_table.qza --output-path phyloseq


Exported dada2/clustered_tab.qza as BIOMV210DirFmt to directory phyloseq
Exported dada2/seqs.qza as DNASequencesDirectoryFormat to directory phyloseq
Exported dada2/tax_table.qza as TSVTaxonomyDirectoryFormat to directory phyloseq
